# A Dual Approach to Holistic Regression
## March 4th, 2021

In [1]:
using Random, Distributions
using LinearAlgebra
using Gurobi, JuMP
using DataFrames
using CSV
using StatsBase
using Plots
using ProgressBars
using Optim

In [2]:
gurobi_env = Gurobi.Env()

function create_gurobi_model(; TimeLimit=-1, LogFile=nothing)
    model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env)));
    if TimeLimit >= 0
        println("Set Gurobi TimeLimit.")
        set_optimizer_attribute(model, "TimeLimit", TimeLimit)
    end
    if LogFile != nothing
        println("LogFile: $(LogFile).")
        set_optimizer_attribute(model, "LogFile", LogFile)
    else
        set_optimizer_attribute(model, "OutputFlag", 0)
    end
    set_optimizer_attribute(model, "NumericFocus", 3)
    return model
end;


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


____
## 0. Utils

In [70]:
function write_list(file_path, l)
    if length(l) == 0
        return
    end
    open(file_path, "a+") do io
        write(io, "\n")
        for e in l[1:end-1]
            try 
                e = round(e, digits=3)
                catch error end
            write(io, "$(e),")
        end
        e = l[end]
        try 
            e = round(e, digits=3)
        catch error end
        write(io, "$(e)")
    end
    return
end

function write_to_file(file_path, str)
    open(file_path, "a+") do io
        write(io, str)
    end
end

write_to_file (generic function with 1 method)

In [71]:
function get_support(s)
    supp = similar(s, Int)
    count_supp = 1
    
    supp_c = similar(s, Int)
    count_supp_c = 1
    
    @inbounds for i in eachindex(s)
        supp[count_supp] = i
        supp_c[count_supp_c] = i
        is_zero = s[i] < 0.5
        count_supp += !is_zero
        count_supp_c += is_zero
    end
    return resize!(supp, count_supp-1), resize!(supp_c, count_supp_c-1)
end

get_support([0, 0, 1, 1, 0])

([3, 4], [1, 2, 5])

____
## 1. Generate Synthetic Data  

In [4]:
function generate_synthetic_data(n, p, k, NR; seed=-1)
    """
        n = num. of samples
        p = num. of features
        k = num. of non zero coefficients
        NR = noise ratio ~ σ_noise = NR * σ_y_true
    """
    if seed >= 0
        Random.seed!(seed)
    end
    
    # Generate PD matrix
    A = randn(p, p)
    A = A'*A
    Σ = (A' + A)/2
    
    # Generate data X
    d = MvNormal(Σ)
    X = rand(d, n)'I
    
    # Split data
    index_train = 1:floor(Int, 0.5*n)
    index_val = floor(Int, 0.5*n)+1:floor(Int, 0.75*n)
    index_test = floor(Int, 0.75*n)+1:n
    
    X_train = X[index_train,:]
    X_val = X[index_val,:]
    X_test = X[index_test,:]
    
    # Center
    μ_train = [mean(X_train[:, j]) for j=1:p]
    for j=1:p
         X_train[:,j] = X_train[:,j] .- μ_train[j]
         X_val[:,j] = X_val[:,j] .- μ_train[j]
         X_test[:,j] = X_test[:,j] .- μ_train[j]
    end
    
    # Scale
    σ_train = [norm(X_train[:, j]) for j=1:p]
    for j=1:p
         X_train[:,j] = X_train[:,j]/σ_train[j]
         X_val[:,j] = X_val[:,j] ./ σ_train[j]
         X_test[:,j] = X_test[:,j] ./ σ_train[j]
    end
    
    # Generate β
    β = zeros(p)
    for j=1:k
        β[floor(Int, j*p/k)] = 1.0*rand([-1, 1])
    end
    
    # Noise
    ϵ = rand(Normal(0, std(X*β)*NR), n)
    
    # Target
    y_train = X_train*β + ϵ[index_train]
    y_val = X_val*β + ϵ[index_val]
    y_test = X_test*β + ϵ[index_test]
            
    return  (X_train, y_train), (X_val, y_val), (X_test, y_test), β
end

function get_t_α(n, p, α)
    return quantile(TDist(n-p), 1 - α/2)
end

function get_σ_X(X, y, γ)
    n, p = size(X)
    
    # Estimator σ
    M_inv = inv(I/γ + X'X)
    σ_tilde = sqrt((y'*(I - X*M_inv*X')*y)/(n-p))
    σ_X = σ_tilde * sqrt.(diag(M_inv))
    
    return σ_X
end

function get_R2(y_pred, y_true, y_train)
    SS_res = norm(y_true .- y_pred)
    SS_tot = norm(y_true .- mean(y_train))
    return 1 - (SS_res/SS_tot)^2
end

;

## 2. Compute inner problems and gradients

### a. Compute g_s

In [5]:
function g_s(D_s, b_s, σ_X_s; GD=true)
    
    # Get length of support of s
    l = length(b_s)
    if l==0
        return zeros(0), 0.0
    end
    
    # Initial solution
    λ_s0 = zeros(l) .+ 1.0

    # Objective and gradient
    function fg!(F, G, λ_s)
        
        μ_s = λ_s .+ b_s
        β_s = D_s*μ_s
        
        if G != nothing
            G .= β_s .- σ_X_s
        end
        
        if F != nothing
            return -λ_s'σ_X_s + 0.5*μ_s'β_s
        end
    end
    
    lower = zeros(l)
    upper = [Inf for _ in 1:l]

    res = Optim.optimize(Optim.only_fg!(fg!), lower, upper, λ_s0, 
        Fminbox(GD ? GradientDescent() : LBFGS()))

    return Optim.minimizer(res), - Optim.minimum(res)
    
end;

In [6]:
function g_s_gurobi(D_s, b_s, σ_X_s, model)
    
    # Get length of support of s
    l = length(b_s)
    if l==0
        return zeros(0), 0.0
    end

    λ_s = model[:λ][1:l]
    μ_s = λ_s .+ b_s
    β_s = D_s*μ_s
    
    @objective(model, Max, λ_s'σ_X_s - 0.5*μ_s'β_s)
    
    optimize!(model)
    
    value.(λ_s), objective_value(model)
end;

In [7]:
function g_gurobi(supp, Z, D, b, σ_X, model)

    # Create DZ once
    DZ = D*Z
    
    λ = model[:λ]
    μ = b + λ

    @objective(model, Max, λ'*Z*σ_X - 0.5μ'*DZ*μ)
    
    optimize!(model)
    
    value.(λ)[supp], objective_value(model)
end;

### b. Compute ∇g_s

In [8]:
function ∇g_s(supp, supp_c, b, M, λ_s, D_s, σ_X_s)
    
    β_s = D_s*(b[supp] .+ λ_s)
  
    grad = zeros(length(b))
    grad[supp] = λ_s .* σ_X_s - (β_s .^ 2)/(2γ)
    grad[supp_c] = - 0.5*γ*(b[supp_c] - M[supp_c, supp]*β_s).^2
    
    return grad
    
end

∇g_s (generic function with 1 method)

In [9]:
function ∇g(supp, D, b, λ_s, σ_X)
    
    λ = zeros(length(b))
    λ[supp] = λ_s
    
    grad = λ .* σ_X - ((D'*(b + λ)).^ 2)/(2γ)

    return grad
    
end

∇g (generic function with 1 method)

_____
## 3. Compare speed 

### /!\ t_α is already in σ_X /!\

In [10]:
# Parameters
n_train = 10000
n = 2*n_train
p = 100
k = 10
NR = 0.001
α = 0.05
t_α = get_t_α(n, p, α)
γ = 1.0

# Generate data
(X_p, y), _, _, β_true = generate_synthetic_data(n, p, k, NR, seed=42);
σ_X_p = t_α * get_σ_X(X_p, y, γ); #t_α is already in σ_X

# Compute data in p dimensions
M_p = X_p'X_p
b_p = X_p'y

# Compute data in 2p dimensions
M = [M_p -M_p; -M_p  M_p]
b = [b_p; -b_p];
σ_X = [σ_X_p; σ_X_p] ;

In [11]:
# Create s
s_true = vcat(β_true .> 0, β_true .< 0) .* 1 
supp, supp_c = get_support(s_true)

# Get projected variables
b_s = b[supp];
σ_X_s = σ_X[supp];
D_s = inv(I/γ + M[supp, supp]);

# Create model for g_s_gurobi
model_inner_g_s = create_gurobi_model();
@variable(model_inner_g_s, λ[1:k] >= 0)

# Create model for g_gurobi
model_inner_g = create_gurobi_model();
@variable(model_inner_g, λ[1:2p] >= 0);

Z = Diagonal(s_true);
D = inv(I/γ + Z*M);

### Compare g_s

In [12]:
λ_s_GD, g_s_GD = g_s(D_s, b_s, σ_X_s; GD=true)
λ_s_LBFGS, g_s_LBFGS = g_s(D_s, b_s, σ_X_s; GD=false)
λ_s_guro_s, g_s_guro_s = g_s_gurobi(D_s, b_s, σ_X_s, model_inner_g_s)
λ_s_guro, g_s_guro = g_gurobi(supp, Z, D, b, σ_X, model_inner_g)

# Compare objective values
println("GD: ",g_s_GD)
println("LBFGS: ",g_s_LBFGS)
println("Gurobi (g_s): ", g_s_guro_s)
println("Gurobi (g): ", g_s_guro)

# Compare λ
hcat(λ_s_GD, λ_s_LBFGS, λ_s_guro_s, λ_s_guro)

GD: -2.3206033744370003
LBFGS: -2.3206033744370003
Gurobi (g_s): -2.320603381830274
Gurobi (g): -2.320603381830274


10×4 Array{Float64,2}:
 1.95201e-21  1.95788e-21  1.30085e-9   1.30085e-9 
 2.59568e-21  2.6034e-21   2.28347e-9   2.28347e-9 
 1.99153e-21  1.99752e-21  1.37628e-9   1.37628e-9 
 2.43976e-21  2.44704e-21  2.06563e-9   2.06563e-9 
 2.44601e-21  2.45331e-21  2.0521e-9    2.0521e-9  
 2.51394e-21  2.52144e-21  2.17646e-9   2.17646e-9 
 2.38831e-21  2.39544e-21  1.99519e-9   1.99519e-9 
 2.38522e-21  2.39234e-21  1.95746e-9   1.95746e-9 
 2.25862e-21  2.26539e-21  1.76581e-9   1.76581e-9 
 1.70973e-21  1.71487e-21  9.54728e-10  9.54728e-10

In [13]:
# Computational time for D_s compared to D (given s)

total_time_D_s = 0
total_time_D = 0
for _ in 1:1000
    
    total_time_D_s += @elapsed begin 
        supp, supp_c = get_support(s_true)
        b_s = b[supp];
        σ_X_s = σ_X[supp];
        D_s = inv(I/γ + M[supp, supp]);
    end
    
    total_time_D += @elapsed begin
        Z = Diagonal(s_true);
        D = inv(I/γ + Z*M);
    end
end

println("Compute D_s: ", total_time_D_s)
println("Compute D: ", total_time_D)

Compute D_s: 0.019539329000000022
Compute D: 2.4000577459999977


In [14]:
# Computational time of g_s and g for given D_s or D

total_time_GD = 0
total_time_LBFGS = 0
total_time_Gurobi_g_s = 0
total_time_Gurobi_g = 0

for _ in 1:1000
    total_time_GD += @elapsed g_s(D_s, b_s, σ_X_s; GD=true)
    total_time_LBFGS += @elapsed g_s(D_s, b_s, σ_X_s; GD=false)
    total_time_Gurobi_g_s += @elapsed g_s_gurobi(D_s, b_s, σ_X_s, model_inner_g_s)
    total_time_Gurobi_g += @elapsed g_gurobi(supp, Z, D, b, σ_X, model_inner_g)
end

println("Optim.jl + GD: ",total_time_GD)
println("Optim.jl + LBFGS: ", total_time_LBFGS)
println("Gurobi g_s (model created outside): ", total_time_Gurobi_g_s)
println("Gurobi g (model created outside): ", total_time_Gurobi_g)

Optim.jl + GD: 0.4202910600000003
Optim.jl + LBFGS: 0.7197528050000007
Gurobi g_s (model created outside): 1.127380273999999
Gurobi g (model created outside): 34.22509962799998


### Compare ∇g_s

In [15]:
# Compare the gradient for different optimal lambdas

∇g_s_guro = ∇g_s(supp, supp_c, b, M, λ_s_guro, D_s, σ_X_s)
∇g_s_GD = ∇g_s(supp, supp_c, b, M, λ_s_GD, D_s, σ_X_s)
∇g_guro = ∇g(supp, D, b, λ_s_guro, σ_X)

println("|| ∇g_s_guro - ∇g_s_GD || =  ", norm(∇g_s_guro - ∇g_s_GD))
println("|| ∇g_s_guro - ∇g_guro || =  ", norm(∇g_s_guro - ∇g_guro))
hcat(∇g_s_GD, ∇g_s_guro, ∇g_guro)[1:10, :]

|| ∇g_s_guro - ∇g_s_GD || =  1.99051676927861e-9
|| ∇g_s_guro - ∇g_guro || =  2.969085093041426e-16


10×3 Array{Float64,2}:
 -0.00769123   -0.00769123   -0.00769123 
 -0.00558403   -0.00558403   -0.00558403 
 -0.000134192  -0.000134192  -0.000134192
 -0.00622613   -0.00622613   -0.00622613 
 -0.000325473  -0.000325473  -0.000325473
 -0.00778968   -0.00778968   -0.00778968 
 -0.000177199  -0.000177199  -0.000177199
 -0.000152214  -0.000152214  -0.000152214
 -0.000558218  -0.000558218  -0.000558218
 -0.146553     -0.146553     -0.146553   

In [18]:
# Computational time for D_s compared to D (given s)

total_time_∇g_s = 0
total_time_∇g = 0

for _ in 1:1000
    
    total_time_∇g_s += @elapsed begin 
        ∇g_s(supp, supp_c, b, M, λ_s_GD, D_s, σ_X_s)
    end
    
    total_time_∇g += @elapsed begin
        ∇g(supp, D, b, λ_s_GD, σ_X)
    end
end

println("Compute ∇g_s: ", total_time_∇g_s)
println("Compute ∇g: ", total_time_∇g)

Compute ∇g_s: 0.02166596299999999
Compute ∇g: 0.010549560000000006


_____
## 4. Compute Cutting plane algorithm

In [43]:
function compute_primal(X, y, k, γ, σ_X; TimeLimit=-1, LogFile=nothing)
    
    n, p = size(X)
    
    model = create_gurobi_model(;LogFile=LogFile, TimeLimit=TimeLimit)

    # TODO: change big-M values
    M1 = 1000
    M2 = 1000

    @variable(model, β[i=1:p])
    @variable(model, s[i=1:p], Bin)
    @variable(model, b[i=1:p], Bin)

    @constraint(model, sum(s) <= k)
    
    @constraint(model, [i=1:p], β[i] <= M1*s[i])
    @constraint(model, [i=1:p], β[i] >= -M1*s[i])

    @constraint(model, [i=1:p], β[i]/σ_X[i] + M2*b[i] >= s[i])
    @constraint(model, [i=1:p], -β[i]/σ_X[i] + M2*(1-b[i]) >= s[i])

    @objective(model, Min, 0.5*sum((y - X*β).^2) + (0.5/γ)*sum((β).^2))
        
    JuMP.optimize!(model)
    
    return objective_value(model), value.(β)
end

compute_primal (generic function with 1 method)

In [44]:
function compute_warm_start_primal(X, y, k, γ, σ_X, time_limit; LogFile=nothing)
    
    n, p = size(X)
    
    model = create_gurobi_model(;TimeLimit=time_limit, LogFile=LogFile)

    # TODO: change big-M values
    M1 = 1000
    M2 = 1000

    @variable(model, β[i=1:p])
    @variable(model, s[i=1:p], Bin)
    @variable(model, b[i=1:p], Bin)

    @constraint(model, sum(s) <= k)
    
    @constraint(model, [i=1:p], β[i] <= M1*s[i])
    @constraint(model, [i=1:p], β[i] >= -M1*s[i])

    @constraint(model, [i=1:p], β[i]/σ_X[i] + M2*b[i] >= s[i])
    @constraint(model, [i=1:p], -β[i]/σ_X[i] + M2*(1-b[i]) >= s[i])

    @objective(model, Min, 0.5*sum((y[i] - X[i,:]'β)^2 for i=1:n) + (0.5/γ)* sum(β[j]^2 for j=1:p))
    JuMP.optimize!(model)
    
    s_val = Int.(value.(s))
    b_val = Int.(value.(b))
    
    return vcat(s_val .* (b_val .== 0), s_val .* (b_val .== 1))
end

compute_warm_start_primal (generic function with 1 method)

In [45]:
function compute_dual(X_p, y, k, γ, σ_X_p; LogFile=nothing, WarmStart=nothing, TimeLimit=-1)
    """
    WarmStart ∈ {  nothing, :RidgeStart, :PrimalStart }
    """
    
    # Get dimensions
    n, p = size(X_p)
    
    # Constant
    C = 0.5*y'y #TODO: add it in the objecive in the end
 
    # Compute data in p dimensions
    M_p = X_p'X_p
    b_p = X_p'y

    # Compute data in 2p dimensions
    M = [M_p -M_p; -M_p  M_p]
    b = [b_p; -b_p];
    σ_X = [σ_X_p; σ_X_p] ;
    
    # Outer problem
    miop = create_gurobi_model(;LogFile=LogFile, TimeLimit=TimeLimit)
    @variable(miop, s[1:2p], Bin)
    @variable(miop, t >= -C)
    @constraint(miop, sum(s) <= k)
    @constraint(miop, [i=1:p], s[i]+s[p+i]<=1)
    
    # Initial solution
    s0 = zeros(2p) #TODO: change this
    
    # a. First k values
    s0[1:k] .= 1
    
    # b. Ridge regression
    if (WarmStart == :RidgeStart)
        println("Ridge Warm Start.")
        β_ridge = inv(I/γ + M_p)*b_p
        s0[findall(x -> x>0, β_ridge)] .= 1.0
        s0[findall(x -> x<0, β_ridge) .+ p] .= 1.0
    end
    
    # c. Primal solution + Time limit
    if (WarmStart == :PrimalStart)
        println("Primal Warm Start.")
        s0 = compute_warm_start_primal(X_p, y, k, γ, σ_X, 20; LogFile=LogFile)
    end

    # Initial cut
    supp, supp_c = get_support(s0)
    
    D_s, b_s, σ_X_s = inv(I/γ + M[supp, supp]), b[supp], σ_X[supp]
    
    λ_s0, g_s0 = g_s(D_s, b_s, σ_X_s; GD=true)
    ∇g_s0 = ∇g_s(supp, supp_c, b, M, λ_s0, D_s, σ_X_s)
    
    @constraint(miop, t >= g_s0 + dot(∇g_s0, s - s0))
    @objective(miop, Min, t + C)
    
    # Cutting planes    
    function outer_approximation(cb_data)
        
        s_val = [callback_value(cb_data, s[i]) for i=1:2p]
        
        supp, supp_c = get_support(s_val)

        D_s, b_s, σ_X_s = inv(I/γ + M[supp, supp]), b[supp], σ_X[supp]

        λ_s_val, g_s_val = g_s(D_s, b_s, σ_X_s; GD=true)
        ∇g_s_val = ∇g_s(supp, supp_c, b, M, λ_s_val, D_s, σ_X_s)
        
        con = @build_constraint(t >= g_s_val + dot(∇g_s_val, s - s_val))
        MOI.submit(miop, MOI.LazyConstraint(cb_data), con)
        
    end
    
    MOI.set(miop, MOI.LazyConstraintCallback(), outer_approximation)
    JuMP.optimize!(miop)
    
    
    s_opt = JuMP.value.(miop[:s])
    supp, supp_c = get_support(s_opt)
    
    D_s, b_s, σ_X_s = inv(I/γ + M[supp, supp]), b[supp], σ_X[supp]
    
    λ_s_opt, g_s_opt = g_s(D_s, b_s, σ_X_s; GD=true)
    
    β = zeros(2p)
    β[supp] = D_s*(λ_s_opt + b_s)
    

    return objective_value(miop), β[1:p] - β[p+1:end]
end

compute_dual (generic function with 1 method)

In [33]:
@time primal_obj, β_primal = compute_primal(X_p, y, k, γ, σ_X_p; LogFile="nouveau_debug.txt")

LogFile: nouveau_debug.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 401 rows, 300 columns and 1100 nonzeros
Model fingerprint: 0x9f08118d
Model has 5050 quadratic objective terms
Variable types: 100 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e-02, 1e+00]
  QObjective range [5e-06, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+03]
Found heuristic solution: objective 9.3237205
Presolve time: 0.01s
Presolved: 401 rows, 300 columns, 1100 nonzeros
Presolved model has 5050 quadratic objective terms
Variable types: 100 continuous, 200 integer (200 binary)

Root relaxation: objective 6.418095e+00, 586 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.41809    0  154    9.32372    6.41809  31.2%     -    0s
H    0     0

(7.003117132664984, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.541392  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.43268])

In [34]:
@time dual_obj, β_dual = compute_dual(X_p, y, k, γ, σ_X_p; LogFile="nouveau_debug.txt")

LogFile: nouveau_debug.txt.
Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 102 rows, 201 columns and 601 nonzeros
Model fingerprint: 0x60fdbfc0
Variable types: 1 continuous, 200 integer (200 binary)
Coefficient statistics:
  Matrix range     [2e-07, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [9e+00, 9e+00]
  RHS range        [2e-01, 1e+01]
Presolve time: 0.00s
Presolved: 102 rows, 201 columns, 601 nonzeros
Variable types: 1 continuous, 200 integer (200 binary)

Root relaxation: objective 4.903414e+00, 21 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    4.97558    0    2          -    4.97558      -     -    0s
H    0     0                       8.2686682    4.97558  39.8%     -    0s
H    0     0                       7.4213394    4.97558  33.0%     -    0s
     0     2    6.81718    0    8    7.42

(7.003117132664965, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.541392  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.43268])

In [36]:
hcat(β_true, β_primal, β_dual)

100×3 Array{Float64,2}:
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 1.0  0.541392  0.541392
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 ⋮                      
 0.0  0.0       0.0     
 1.0  0.44302   0.44302 
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 0.0  0.0       0.0     
 1.0  0.43268   0.43268 

_____
## 5. Experiences

In [72]:
# Parameters
n_train = 10000
n = 2*n_train
NR = 0.001
α = 0.05;
t_max = 30*60 # 30 min max per solving

1800

In [ ]:
csv_path = "results/2021_03_06_results.csv"
logfile = "results/2021_03_06_logs.txt"

write_list(csv_path, ["Algo", "Seed", "n", "p", "k_true", "k", "γ", 
                      "NR", "α", "R2", "OR2", "t_algo", "t_data", "t_variance"])

for seed ∈ [1997, 1998]
    for p ∈ [100, 150, 200, 300]

        # True sparsity
        k_true = div(p, 10)

        # Generate data
        t_data = @elapsed (X_train, y_train), _, (X_test, y_test), β_true = generate_synthetic_data(n, p, k_true, NR, seed=seed);

        # Significance
        t_variance = @elapsed t_α = get_t_α(n_train, p, α)
        
        # Robustness
        for γ ∈ [1.0, 10.0, 100.0]
            
            # Variance
            t_variance += @elapsed σ_X = t_α * get_σ_X(X_train, y_train, γ)
        
            # Estimated sparsity
            Random.seed!(seed)
            k = rand(5:15)
            
            # Dual
            t_dual = @elapsed obj_dual, β_dual = compute_dual(X_train, y_train, k, γ, σ_X; LogFile=logfile, TimeLimit=t_max);
            R2_dual = get_R2(X_train*β_dual, y_train, y_train)
            OR2_dual = get_R2(X_test*β_dual, y_test, y_train)
            write_list(csv_path, ["dual", seed, n_train, p, k_true, k, γ, 
                                  NR, α, R2_dual, OR2_dual, t_dual, t_data, t_variance])
            
            # Primal
            t_primal = @elapsed obj_primal, β_primal = compute_primal(X_train, y_train, k, γ, σ_X; LogFile=logfile, TimeLimit=t_max);
            R2_primal = get_R2(X_train*β_primal, y_train, y_train)
            OR2_primal = get_R2(X_test*β_primal, y_test, y_train)
            write_list(csv_path, ["primal", seed, n_train, p, k_true, k, γ, 
                                  NR, α, R2_primal, OR2_primal, t_primal, t_data, t_variance])
            
        end
    end
end

Set Gurobi TimeLimit.
LogFile: results/2021_03_06_logs.txt.
